In [111]:
ng = {}

def create_ngram(states):
    
    for cmd in states:
        path = ng
        for i in range(len(cmd)):
            if cmd[i] in path:
                path[cmd[i]]["count"] += 1
            else:
                path[cmd[i]] = {
                    "type":"No value",
                    "count":1,
                    "cmd":cmd[i],
                    "next":{}
                }

            path = path[cmd[i]]["next"]
        



In [112]:
n = 3
state = []

def create_state(cmd):
    for i in range(len(cmd)):
        s = []
        for j in range(i,min(i+n,len(cmd))):
            s.append(cmd[j])
        
        s = [v for v in s if len(v) >= 2]
        if len(s) >= 2:
            state.append(s)

with open("cmd_data.txt","r") as f:
    for row in f:
        cmd = row.split()
        create_state(cmd)

In [113]:
create_ngram(state)

In [122]:
import json
model = json.dumps(ng,ensure_ascii=False,indent=2)
with open("model.json","w") as f:
    f.write(model)